In [1]:
# | default_exp download_bot_datasets


This Notebook collects scripts to download various benchmark-datasets. Most of them can be found on huggingface.

| Name                                                                                   | Description                                                                                                                                                                                                                   |
|----------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| [GSM8K](https://huggingface.co/datasets/openai/gsm8k)                                  | A dataset of 8,000 high-quality grade-school math problems, used to benchmark models on mathematical reasoning and problem-solving skills.                                                                                     |
| [AmbigNQ](https://nlp.cs.washington.edu/ambigqa/)                         | A subset of Natural Questions focusing on ambiguous questions with multiple valid answers, useful for evaluating how well models handle ambiguity in queries.                                                                  |
| [HumanEval](https://huggingface.co/datasets/openai/openai_humaneval)                                      | A dataset for evaluating the correctness of Python code generated by models, providing function signatures, docstrings, and test cases to assess the model's programming capabilities.                                          |
| [Game of 24](https://github.com/princeton-nlp/tree-of-thought-llm)                            | A mathematical reasoning dataset based on the game where players manipulate four numbers with basic arithmetic operations to reach 24, testing a model's numerical reasoning and calculation skills.                            |
| [Python Programming Puzzles (P3)](https://github.com/microsoft/PythonProgrammingPuzzles)| A collection of Python programming challenges designed to probe the limits of programming comprehension and reasoning, valuable for evaluating logic, problem-solving, and programming ability.                                  |
| [BIG-Bench Hard (BBH)](https://github.com/google/BIG-bench)                  | A subset of the BIG-Bench benchmark focusing on particularly challenging tasks for language models, spanning diverse categories like reasoning, linguistics, and abstract thought.                                               |

In [2]:
# | export

# load_dataset from the datasets library: Facilitates loading datasets from the Hugging Face Hub. (pip install datasets)
from datasets import load_dataset
from pathlib import Path
from typing import Optional
import requests
import zipfile
import io

/Users/oliverpfante/Documents/agentic/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# | export

def download_file_from_url(
    url: str,
    filepath: Path,
):
    """
    Download a file from a URL.

    Parameters
    ----------
    url
        URL of the file to download.
    filepath
        Path where the downloaded file will be saved.

    Returns
    -------
    None
        This function performs file download but does not return any value.
    """
    if not filepath.exists():
        print(f"Downloading file from {url}...")
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(filepath, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"File downloaded and saved to {filepath}.")
    else:
        print(f"File {filepath} already exists. Skipping download.")

In [4]:
# | export

def download_from_huggingface(data_path: Path, path: str, name: Optional[str] = None):
    """
    Download a dataset from Hugging Face and save its splits to a specified directory.

    Parameters
    ----------
    data_path : Path
        The root directory where the dataset will be saved.
    path : str
        The dataset identifier on Hugging Face (e.g., "openai_humaneval").
    name :
        An additional name of the dataset. Will be used to create an extra directory. If None, no extra directory is created.

    Returns
    -------
    None
        This function downloads the dataset and saves it locally.
    """
    # Construct the target directory
    target_path = data_path / path
    if name:
        target_path = target_path / name

    # Check if the target directory exists and contains files
    if target_path.exists() and any(target_path.iterdir()):
        print(f"Dataset already exists in {target_path}. Skipping download.")
        return

    # Load the dataset from Hugging Face
    dataset = load_dataset(path=path, name=name)

    # Ensure the target directory exists
    target_path.mkdir(parents=True, exist_ok=True)

    # Save each split of the dataset to the target directory
    for split in dataset.keys():
        split_dataset = dataset[split]
        split_path = target_path / f"{split}.jsonl"
        split_dataset.to_json(split_path)
        print(f"Saved {split} split to {split_path}")

In [5]:
data_path = Path('data')

In [6]:
# | export

def download_gsm8k(
    data_path: Path,
):
    """
    Download the GSM8k dataset into a specified folder.

    Parameters
    ----------
    path
        Directory path where the GSM8k dataset will be saved. If the directory does not exist, it will be created.

    Returns
    -------
    None
        This function downloads the dataset and saves it locally but does not return any value.
    """
    download_from_huggingface(data_path, "openai/gsm8k", "main")

In [7]:
download_gsm8k(data_path)

Dataset already exists in data/openai/gsm8k/main. Skipping download.


In [8]:
# | export

def download_ambignq(
    path: Path,
):
    """
    Download the AmbigNQ dataset into a specified folder.

    Parameters
    ----------
    path
        Directory path where the AmbigNQ dataset will be saved. If the directory does not exist, it will be created.

    Returns
    -------
    None
        This function downloads the dataset, extracts its contents, and saves them locally but does not return any value.
    """
    # URL for AmbigNQ dataset
    url = "https://nlp.cs.washington.edu/ambigqa/data/ambignq_light.zip"

    # Convert path to Path object and create directory if it doesn't exist
    ambignq_path = path / "ambignq"
    ambignq_path.mkdir(parents=True, exist_ok=True)

    # Download the ZIP file
    print(f"Downloading ZIP file from {url}...")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    # Open the ZIP file in memory and extract its contents
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
        print("Unpacking ZIP file...")
        zip_file.extractall(ambignq_path)
    print(f"Unpacked contents to {ambignq_path}")

In [9]:
download_ambignq(data_path)

Unpacking ZIP file...
Unpacked contents to data/ambignq


In [10]:
# | export

def download_humaneval(data_path: Path):
    """
    Download the HumanEval dataset and save it to the specified directory.

    Parameters
    ----------
    data_path :
        The directory path where the HumanEval dataset will be saved. If the directory does not exist, it will be created.

    Returns
    -------
    None
        This function downloads the dataset and saves it locally but does not return any value.
    """
    download_from_huggingface(data_path, path="openai_humaneval")

In [11]:
download_humaneval(data_path)

Dataset already exists in data/openai_humaneval. Skipping download.


In [12]:
# | export

def download_game_of_24(data_path: Path):
    """
    Download the Game of 24 dataset using Hugging Face's `load_dataset` and save it to the specified directory.

    Parameters
    ----------
    path : Path
        The directory path where the Game of 24 dataset will be saved. If the directory does not exist, it will be created.

    Returns
    -------
    None
        This function downloads the dataset and saves it locally but does not return any value.
    """
    download_from_huggingface(data_path, path="nlile/24-game")

In [13]:
download_game_of_24(data_path)

Dataset already exists in data/nlile/24-game. Skipping download.


In [14]:
# | export

def download_python_programming_puzzles(path: Path):
    """
    Download the Python Programming Puzzles (P3) dataset and save it to the specified directory.

    Parameters
    ----------
    path : Path
        The directory path where the P3 dataset will be saved. If the directory does not exist, it will be created.

    Returns
    -------
    None
        This function downloads the dataset and saves it locally but does not return any value.
    """
    # Ensure the target directory exists
    p3_path = path / "python_programming_puzzles"
    p3_path.mkdir(parents=True, exist_ok=True)

    # URL for the P3 dataset (update this URL if needed)
    url = "https://github.com/microsoft/PythonProgrammingPuzzles/blob/main/puzzles/puzzles.json"

    download_file_from_url(url, p3_path / 'puzzles.json')

In [15]:
download_python_programming_puzzles(data_path)

File data/python_programming_puzzles/puzzles.json already exists. Skipping download.


In [16]:
# | export

def download_bhb_tasks(data_path: Path):
    """
    Download specific tasks from Google's BIG-Bench Hard (BBH) dataset and save them to the specified directory.

    Parameters
    ----------
    data_path : Path
        The root directory where the BBH tasks will be saved. Each task will be stored in a subdirectory 
        based on its respective name.

    Returns
    -------
    None
        This function downloads the BBH tasks and saves them locally, with separate directories for each task.
    """
    for path, name in [
        ("maveriq/bigbenchhard", "boolean_expressions"),
        ("maveriq/bigbenchhard", "causal_judgement"),
        ("maveriq/bigbenchhard", "date_understanding"),
        ("maveriq/bigbenchhard", "word_sorting")
    ]:
        download_from_huggingface(data_path, path, name)

In [17]:
download_bhb_tasks(data_path)

Dataset already exists in data/maveriq/bigbenchhard/boolean_expressions. Skipping download.
Dataset already exists in data/maveriq/bigbenchhard/causal_judgement. Skipping download.
Dataset already exists in data/maveriq/bigbenchhard/date_understanding. Skipping download.
Dataset already exists in data/maveriq/bigbenchhard/word_sorting. Skipping download.


In [18]:
# | hide
from nbdev.showdoc import *

In [19]:
# | hide
import nbdev

nbdev.nbdev_export()